In [ ]:
import os
import shutil
import sys
import torch
from google.colab import drive

# Mount Drive
drive.mount('/content/drive')

# Restore Dataset
if not os.path.exists('/content/dataset/project_data/cityscapes'):
    print("Restoring Cityscapes...")
    zip_path_1 = '/content/drive/MyDrive/semseg/project_data.zip'
    zip_path_2 = '/content/drive/MyDrive/project_data.zip'

    if os.path.exists(zip_path_1):
        shutil.unpack_archive(zip_path_1, '/content/dataset')
    elif os.path.exists(zip_path_2):
        shutil.unpack_archive(zip_path_2, '/content/dataset')
    else:
        print("Dataset zip not found. Please check paths.")

Mounted at /content/drive
Restoring Cityscapes...


In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import os
import sys

# --- 1. SETUP & IMPORTS ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🚀 Training on device: {device}")

# Import the model from the specific file we found earlier
try:
    from models.bisenet.build_bisenet import BiSeNet
except ImportError:
    # Safety fallback if python path is different
    sys.path.append('/content/MLDL2024_project1')
    from models.bisenet.build_bisenet import BiSeNet

# --- 2. INITIALIZE MODEL ---
# We use 'context_path' because that is what your build_bisenet.py requires
print("Initializing BiSeNet with ResNet18...")
try:
    model = BiSeNet(num_classes=19, context_path='resnet18')
except TypeError:
    # Fallback if the argument name is different in some versions
    model = BiSeNet(n_classes=19, backbone='resnet18')

model.to(device)

# --- 3. OPTIMIZER & LOSS ---
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss(ignore_index=255) # 255 is the 'void' class to ignore

# --- 4. CHECKPOINT SYSTEM (Saves to Google Drive) ---
checkpoint_path = '/content/drive/MyDrive/bisenet_cityscapes_checkpoint.pth'
start_epoch = 0
num_epochs = 50

if os.path.exists(checkpoint_path):
    print(f"🔄 Found checkpoint at {checkpoint_path}. Resuming...")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming from Epoch {start_epoch}")
else:
    print("🆕 Starting fresh training...")

# --- 5. TRAINING LOOP ---
print(f"Starting training for {num_epochs} epochs...")

for epoch in range(start_epoch, num_epochs):
    model.train()
    running_loss = 0.0

    # train_loader must be defined from the previous step
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass (BiSeNet outputs a tuple: main_loss, aux1, aux2)
        outputs = model(images)

        if isinstance(outputs, tuple):
            # Calculate loss for all 3 outputs for better training stability
            loss_main = criterion(outputs[0], labels)
            loss_aux1 = criterion(outputs[1], labels)
            loss_aux2 = criterion(outputs[2], labels)
            loss = loss_main + 0.1 * loss_aux1 + 0.1 * loss_aux2
        else:
            loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Print status every 50 batches
        if i % 50 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Step [{i}/{len(train_loader)}] Loss: {loss.item():.4f}")

    # SAVE CHECKPOINT
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': running_loss
    }, checkpoint_path)

    print(f"✅ Epoch {epoch+1} Complete. Checkpoint saved.")

print("🎉 Training Finished!")

🚀 Training on device: cuda
Initializing BiSeNet with ResNet18...
🆕 Starting fresh training...
Starting training for 50 epochs...
Epoch [1/50] Step [0/197] Loss: 4.8958
Epoch [1/50] Step [50/197] Loss: 0.9754
Epoch [1/50] Step [100/197] Loss: 0.8764
Epoch [1/50] Step [150/197] Loss: 0.7089
✅ Epoch 1 Complete. Checkpoint saved.
Epoch [2/50] Step [0/197] Loss: 0.6278
Epoch [2/50] Step [50/197] Loss: 0.5402
Epoch [2/50] Step [100/197] Loss: 0.5668
Epoch [2/50] Step [150/197] Loss: 0.6074
✅ Epoch 2 Complete. Checkpoint saved.
Epoch [3/50] Step [0/197] Loss: 0.4910
Epoch [3/50] Step [50/197] Loss: 0.6091
Epoch [3/50] Step [100/197] Loss: 0.5502
Epoch [3/50] Step [150/197] Loss: 0.4078
✅ Epoch 3 Complete. Checkpoint saved.
Epoch [4/50] Step [0/197] Loss: 0.3835
Epoch [4/50] Step [50/197] Loss: 0.4872
Epoch [4/50] Step [100/197] Loss: 0.4115
Epoch [4/50] Step [150/197] Loss: 0.4065
✅ Epoch 4 Complete. Checkpoint saved.
Epoch [5/50] Step [0/197] Loss: 0.3774
Epoch [5/50] Step [50/197] Loss: 0.3

In [ ]:
import torch
import numpy as np
import time
import torch.nn.functional as F
# Re-import to ensure we can rebuild the model
import sys
try:
    from models.bisenet.build_bisenet import BiSeNet
except ImportError:
    sys.path.append('/content/MLDL2024_project1')
    from models.bisenet.build_bisenet import BiSeNet

# --- 1. HARD RESET: RELOAD MODEL FROM CHECKPOINT ---
# This wipes out the "AttributeError" caused by thop
print("🔄 Reloading model from checkpoint to clear errors...")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiSeNet(num_classes=19, context_path='resnet18')
model.to(device)

# Load the weights you just trained
checkpoint_path = '/content/drive/MyDrive/bisenet_cityscapes_checkpoint.pth'
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()
print("✅ Model restored successfully!")

# --- 2. CALCULATE mIoU (Accuracy) ---
print("1️⃣  Evaluating Accuracy (mIoU)...")
def fast_hist(a, b, n):
    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n**2).reshape(n, n)

def per_class_iu(hist):
    with np.errstate(divide='ignore', invalid='ignore'):
        return np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))

n_classes = 19
hist = np.zeros((n_classes, n_classes))

with torch.no_grad():
    for i, (images, labels) in enumerate(val_loader):
        images = images.to(device)
        labels = labels.numpy()

        outputs = model(images)
        if isinstance(outputs, tuple): outputs = outputs[0]

        outputs = F.interpolate(outputs, size=(512, 1024), mode='bilinear', align_corners=True)
        preds = outputs.data.max(1)[1].cpu().numpy()

        hist += fast_hist(labels.flatten(), preds.flatten(), n_classes)

ious = per_class_iu(hist)
mIoU = np.nanmean(ious) * 100
print(f"🏆 mIoU Result: {mIoU:.2f}%")

# --- 3. CALCULATE LATENCY & FLOPs (Safe Mode) ---
print("2️⃣  Calculating Technical Metrics...")
from thop import profile, clever_format

# Prepare input
input_tensor = torch.randn(1, 3, 512, 1024).to(device)

# Measure Latency FIRST
start = time.time()
with torch.no_grad():
    for _ in range(50): _ = model(input_tensor)
end = time.time()
latency_ms = ((end - start) / 50) * 1000
fps = 1000 / latency_ms

# Measure FLOPs LAST (This modifies the model, so we do it at the very end)
flops_count, params_count = profile(model, inputs=(input_tensor, ), verbose=False)
flops_str, params_str = clever_format([flops_count, params_count], "%.3f")

print("\n" + "="*40)
print("📊 FINAL RESULTS FOR TABLE 2")
print("="*40)
print(f"✅ mIoU:       {mIoU:.2f}%")
print(f"✅ Latency:    {latency_ms:.2f} ms")
print(f"✅ FPS:        {fps:.2f}")
print(f"✅ FLOPs:      {flops_str}")
print(f"✅ Params:     {params_str}")
print("="*40)

🔄 Reloading model from checkpoint to clear errors...
✅ Model restored successfully!
1️⃣  Evaluating Accuracy (mIoU)...
🏆 mIoU Result: 54.08%
2️⃣  Calculating Technical Metrics...

📊 FINAL RESULTS FOR TABLE 2
✅ mIoU:       54.08%
✅ Latency:    13.38 ms
✅ FPS:        74.76
✅ FLOPs:      82.957G
✅ Params:     23.231M
